<a href="https://colab.research.google.com/github/nucleosight/nlp/blob/main/Urdu_Word_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fetch Corpus sources

In [1]:
!git clone 'https://github.com/zeerakahmed/makhzan'

Cloning into 'makhzan'...
remote: Enumerating objects: 17388, done.
remote: Counting objects: 100% (389/389), done.
remote: Compressing objects: 100% (375/375), done.
remote: Total 17388 (delta 26), reused 84 (delta 14), pack-reused 16999 (from 1)
Receiving objects: 100% (17388/17388), 130.65 MiB | 10.36 MiB/s, done.
Resolving deltas: 100% (10286/10286), done.
Updating files: 100% (6346/6346), done.


In [ ]:
!pip install gensim

Requirements

In [11]:
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import nltk

# Download necessary NLTK data (if you haven't already)
nltk.download('punkt')
nltk.download('stopwords')



# Function to preprocess the text: tokenize, remove punctuation and stopwords
def preprocess_text(text):
    # Tokenize the text into words
    tokens = word_tokenize(text.lower())

    # Remove punctuation
    tokens = [word for word in tokens if word not in string.punctuation]

    # Remove stopwords
    stop_words = ['اب', 'ابھی', 'اپنا', 'اپنے', 'اپنی', 'اٹھا', 'اس', 'اسے', 'اسی', 'اگر', 'ان', 'انہوں', 'انہی', 'انہیں', 'انھیں', 'او', 'اور', 'اے', 'ایسا', 'ایسے', 'ایسی', 'ایک', 'آ', 'آپ', 'آتا', 'آتے', 'آتی', 'آگے', 'آنا', 'آنے', 'آنی', 'آئے', 'آئی', 'آئیں', 'آیا', 'با', 'بڑا', 'بڑے', 'بڑی', 'بعد', 'بعض', 'بلکہ', 'بہت', 'بھی', 'بے', 'پاس', 'پر', 'پہلے', 'پھر', 'تا', 'تاکہ', 'تب', 'تجھ', 'تجھے', 'تک', 'تم', 'تمام', 'تمہارا', 'تمہارے', 'تمھارے', 'تمہاری', 'تمہیں', 'تمھیں', 'تھا', 'تھے', 'تھی', 'تھیں', 'تو', 'تیری', 'تیرے', 'جا', 'جاتا', 'جاتی', 'جاتے', 'جاتی', 'جانے', 'جانی', 'جاؤ', 'جائے', 'جائیں', 'جب', 'جس', 'جن', 'جنہوں', 'جنہیں', 'جو', 'جیسا', 'جیسے', 'جیسی', 'جیسوں', 'چاہیئے', 'چلا', 'چاہے', 'چونکہ', 'حالاں', 'حالانکہ', 'دو', 'دونوں', 'دوں', 'دے', 'دی', 'دیا', 'دیں', 'دیے', 'دیتا', 'دیتے', 'دیتی', 'دینا', 'دینے', 'دینی', 'دیئے', 'ڈالا', 'ڈالنا', 'ڈالنے', 'ڈالنی', 'ڈالے', 'ڈالی', 'ذرا', 'رکھا', 'رکھتا', 'رکھتے', 'رکھتی', 'رکھنا', 'رکھنے', 'رکھنی', 'رکھے', 'رکھی', 'رہ', 'رہا', 'رہتا', 'رہتے', 'رہتی', 'رہنا', 'رہنے', 'رہنی', 'رہو', 'رہے', 'رہی', 'رہیں', 'زیادہ', 'سا', 'سامنے', 'سب', 'سکتا', 'سو', 'سے', 'سی', 'شاید', 'صرف', 'طرح', 'طرف', 'عین', 'کا', 'کبھی', 'کچھ', 'کہہ', 'کر', 'کرتا', 'کرتے', 'کرتی', 'کرنا', 'کرنے', 'کرو', 'کروں', 'کرے', 'کریں', 'کس', 'کسے', 'کسی', 'کہ', 'کہا', 'کہے', 'کو', 'کون', 'کوئی', 'کے', 'کی', 'کیا', 'کیسے', 'کیوں', 'کیونکہ', 'کیے', 'کئے', 'گا', 'گویا', 'گے', 'گی', 'گیا', 'گئے', 'گئی', 'لا', 'لاتا', 'لاتے', 'لاتی', 'لانا', 'لانے', 'لانی', 'لایا', 'لائے', 'لائی', 'لگا', 'لگے', 'لگی', 'لگیں', 'لو', 'لے', 'لی', 'لیا', 'لیتا', 'لیتے', 'لیتی', 'لیکن', 'لیں', 'لیے', 'لئے', 'مجھ', 'مجھے', 'مگر', 'میرا', 'میرے', 'میری', 'میں', 'نا', 'نہ', 'نہایت', 'نہیں', 'نے', 'ہاں', 'ہر', 'ہم', 'ہمارا', 'ہمارے', 'ہماری', 'ہو', 'ہوا', 'ہوتا', 'ہوتے', 'ہوتی', 'ہوتیں', 'ہوں', 'ہونا', 'ہونگے', 'ہونے', 'ہونی', 'ہوئے', 'ہوئی', 'ہوئیں', 'ہے', 'ہی', 'ہیں', 'و', 'والا', 'والوں', 'والے', 'والی', 'وہ', 'وہاں', 'وہی', 'وہیں', 'یا', 'یعنی', 'یہ', 'یہاں', 'یہی', 'یہیں']

    #set(stopwords.words('urdu'))
    tokens = [word for word in tokens if word not in stop_words]

    return tokens

# Function to read a corpus file and preprocess the text
def read_corpus(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # Preprocess the text
    processed_text = preprocess_text(text)
    return processed_text


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Prepare Text Corpus

In [4]:
# extract urdu text from  every xml file in the directory
import os
import re
import xml.etree.ElementTree as ET
text_directory_path = '/content/makhzan/text'
texts = []
def extract_text_from_xml(file_path):
  """Extracts text from all elements in an XML file except those with the 'annotation' tag."""
  tree = ET.parse(file_path)
  root = tree.getroot()
  text = ""
  for elem in root.iter():
    if elem.tag  in ["title", "p"]:
      text += elem.text or ""
  return text

for filename in os.listdir(text_directory_path):
    if filename.endswith(".xml"):
        file_path = os.path.join(text_directory_path, filename)
        texts.append(extract_text_from_xml(file_path))

In [5]:
# save extracted text into a file
with open('/content/processed_makhzan_text.txt', 'w',
          encoding='utf-8') as f:
  for text in texts:
    f.write(text + '\n')


In [22]:
# Load and preprocess the corpus
corpus_file = '/content/processed_makhzan_text.txt'  # Specify your corpus file here
corpus = [read_corpus(corpus_file)]

# Train Word2Vec model
model = Word2Vec(sentences=corpus, vector_size=50, window=5, min_count=4, workers=8)

# Save the model
model.save('word2vec.model')


In [27]:

# Example usage: Find similar words to a given word
word = 'زمین'  # Replace with any word of your choice
if word in model.wv:
    similar_words = model.wv.most_similar(word)
    print(f"Words similar to '{word}':")
    for sim_word, score in similar_words:
        print(f"{sim_word}: {score}")
else:
    print(f"'{word}' not in vocabulary.")


Words similar to 'زمین':
مہان: 0.5318350195884705
وی: 0.5101794004440308
چمچے: 0.501324474811554
۔یوں: 0.4957681894302368
سفرسے: 0.4939730167388916
کالا،: 0.4938831329345703
لاوا: 0.49330034852027893
بتاریخ: 0.4813132882118225
مہاجنوں: 0.478219211101532
۔سب: 0.4693716764450073


In [18]:
len(model.wv)

211130